<a href="https://colab.research.google.com/github/nobace/investimentos/blob/master/AtualizaDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread
!pip install --upgrade gspread-dataframe

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

import zipfile
import urllib.request
from urllib.request import urlopen

from gspread_dataframe import get_as_dataframe, set_with_dataframe

gc= gspread.authorize(creds)

import pandas as pd

from google.colab import files

!pip install yfinance==0.2.28

import yfinance as yf

from datetime import date
from pandas.tseries.offsets import BDay


planilha = gc.open('Carteiras')

def busca_informes_cvm():
  hoje = (date.today()-BDay(2))

  print(hoje)
  filename = 'inf_diario_fi_{}{:02d}'.format(hoje.year, hoje.month)
  url = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/'+ filename+ '.zip'

  print ("URL:"+url)

  filehandle, _ = urllib.request.urlretrieve(url)
  archive = zipfile.ZipFile(filehandle, 'r')

  resultado = pd.read_csv(archive.open(filename+ '.csv'), sep=';')
  ultimo = resultado.tail(2)
  resultado = resultado[resultado['DT_COMPTC'] == ultimo['DT_COMPTC'].values[0]]

  resultado['VL_QUOTA'] = resultado['VL_QUOTA'].astype(str)
  resultado['VL_QUOTA'] = [x.replace(',', '') for x in resultado['VL_QUOTA']]
  resultado['VL_QUOTA'] = [x.replace('.', ',') for x in resultado['VL_QUOTA']]
  return resultado


def AtualizaPrecos(plan):
  carteira = pd.DataFrame(plan.get_all_records())
  print (carteira)
  carteira.set_index('Código', inplace=True)
  Tickers = carteira.index
  StrT= ".SA ".join(Tickers)
  Suf = ".SA"
  StrT = StrT + Suf

  print ('Papeis;'+ StrT)
  dados_yf = yf.download(tickers=StrT, period='1d')["Adj Close"]


  Valores = dados_yf.tail(n=1)

  i = 2

  for Papel in Tickers:
      Valor = Valores.get(Papel+'.SA').get(0)
      if (pd.isna(Valor) == False):
        print('Atualizando '+Papel+' : '+str(Valor))
        plan.update_cell(i ,4, Valor)
      i = i + 1

def AtualizaUS(plan):
  carteira = pd.DataFrame(plan.get_all_records())
  print (carteira)
  carteira.set_index('Código', inplace=True)
  Tickers = carteira.index
  StrT= " ".join(Tickers)
  Suf = " "
  StrT = StrT + Suf


  dados_yf = yf.download(tickers=StrT, period='1d')["Adj Close"]
  print ('Papeis:'+ StrT)

  Valores = dados_yf.tail(n=1)

  i = 2

  for Papel in Tickers:
      Valor = Valores.get(Papel).get(0)
      if (pd.isna(Valor) == False):
        print('Atualizando '+Papel+' : '+str(Valor))
        plan.update_cell(i ,4, Valor)
      i = i + 1



BaseFIIs = planilha.get_worksheet(2)
print (BaseFIIs)
AtualizaPrecos(BaseFIIs)
BasePrecos = planilha.get_worksheet(0)
AtualizaPrecos(BasePrecos)
BaseUS = planilha.get_worksheet(3)
AtualizaUS(BaseUS)
fundos = busca_informes_cvm()

fundosinv = planilha.get_worksheet(1)

set_with_dataframe(fundosinv, fundos, include_index=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 775.9 kB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 701.4 kB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.38
    Uninstalling yfinance-0.2.38:
      Successfully uninstalled yfinance-0.2.38
<Worksheet 'Base FIIs' id:1273247238>
     Código                         Ação   Tipo         Preço Categoria
0    ABCP11                 FII ABC IMOB     CI  7.000000e+03       FII
1    ALZR11                  FII ALIANZA     CI  1.113000e+04       FII
2    CNES11                                  CI  1.730000e+02       FII
3   BBFI11B                 FII BB PROGR  CI MB  1.249800e+00       FII
4    BBRC11                  FII BB CORP     CI  1.062500e+04       FII
..      ...                          ...    ...